In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy                  # http://scipy.org/
import scipy.odr, scipy.special, scipy.stats 
import scipy.signal as signal

import sympy as sy


import matplotlib.pyplot as plt
%matplotlib inline
params = {'legend.fontsize': 24,
          'figure.figsize': (12, 10),
         'axes.labelsize': 24,
         'axes.titlesize': 24,
         'xtick.labelsize':24,
         'ytick.labelsize':24}
plt.rcParams.update(params)

#Simple sintonizado, una etapa.


from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def Yin_Yout(y):
    yl = y[1,1].real - y[1,1].imag*1j
    for i in range(10):
        yin  = y[0,0] - y[0,1]*y[1,0]/(y[1,1]+yl)
        yg   = yin.real - yin.imag*1j
        yout = y[1,1] - y[0,1]*y[0,1]/(y[0,0]+yg)
        yl   = yout.real - yout.imag*1j
    return (yin, yout)

# Ejercicio con el transistor unilateral


In [11]:
fo = 100e6
BW = 1e6
wo = 2*np.pi*fo
Qo = 150
rg = 100
rl = 1e3
Nt =20
Ytr = np.matrix([ [ 1e-3+5e-12*wo*1j ,  -10e-6-628e-6j  ], 
                  [50e-3-50e-3j   ,  0.1e-3+5e-12*wo*1j ]])




In [13]:
#Calculo de los Q
x = sy.Symbol('x')
ecu = sy.expand((1+x**2)*(1+(1.5*x)**2)-2)
x1 = sy.solve(ecu, x)
Q2 = x1[1]*fo/BW 
Q1 = 1.5 * Q1

print('Q1=', Q1)
print('Q2=', Q2)


#Calculo L1 en la salida suponiendo unilateral

Rext = 1/Ytr[1,1].real/2 
print(Rext)
L1 = Rext*(1/Q2 - 1/Qo )/wo
print('L1=',L1)
print('rp=',Qo*L1*wo)


Q1= 76.5817048399464
Q2= 51.0544698932976
5000.0
L1= 1.02816137654821e-7
rp= 9690.19268180590


In [14]:
# Calculo en la entrada usando el inductor(L1)
Rext = wo*L1/(1/Q1 - 1/Qo )
R = Rext * 2

# relacion de vueltas del generado a tope de bobina
NT1 = (R/rg)**0.5

# relacion de vueltas de la impedancia de entrada del TR a tope de bobina
rintr = 1/Ytr[0,0].real  
NT12   = (R/rintr)**0.5

# Calculo del Capacitor de sintonia en la entrada
Cs1 = 1/(wo**2*L1)


citr = Ytr[0,0].imag/wo  
citrt = citr/NT12**2 
C1  = Cs1 - citrt

print('NT/N1=',NT1)
print('N1=',NT1)

print('NT/(N1+N2)=',NT12)
print('C1=',C1)
      
# calculo de los inductores de la entrada
L1a = L1 /NT1**2      

NT2 = 1/(1/NT12 - 1/NT1)
L1b = L1 /NT2**2

NT3 = 1/(1-1/NT1-1/NT2)
L1c = L1 /NT3**2

print('L1c=',L1c)
print('L1b=',L1b)
print('L1a=',L1a)


NT/N1= 14.2181002332288
N1= 14.2181002332288
NT/(N1+N2)= 4.49615807375741
C1= 2.43891624763712e-11
L1c= 6.21670505260185e-8
L1b= 2.37794094529938e-9
L1a= 5.08602092041147e-10


In [17]:
# calculo del capacitor a la salida
# Calculo del Capacitor de sintonia en la entrada
Cs2 = 1/(wo**2*L1)

cotr = Ytr[1,1].imag/wo  
Ct  = Cs2 - cotr

# divisor capacitivo
Q  = Q2*2
rt = 1/Ytr[1,1].real
N  = (rt/rl)**0.5
Qm = Q/N
print('Qm=' ,Qm, '  (Si es mayor a 10 puedo aproximar)' )

# aproximando!
C3 = Ct*N/(N-1)
C2 = Ct*N




print('C2=',C2)
print('C3=',C3)


Qm= 32.2896819190628   (Si es mayor a 10 puedo aproximar)
C2= 6.20960596062324e-11
C3= 2.87178935203895e-11


# Ejercicio con el transistor neutralizado

In [39]:
fo = 100e6
BW = 1e6
wo = 2*np.pi*fo
Qo = 150
rg = 100
rl = 1e3
Ytr = np.matrix([ [ 1e-3+5e-12*wo*1j ,  -10e-6-628e-6j  ], 
                  [50e-3-50e-3j   ,  0.1e-3+5e-12*wo*1j ]])

print(-Ytr[0,1].imag/wo)
print(-Ytr[1,0].imag/wo)


cp = 4.7e-12
n  = -wo*cp / Ytr[0,1].imag

Yc = np.matrix([ [ wo*cp*1j      ,  wo*cp/n*1j  ], 
                 [ wo*cp/n*1j    ,  wo*cp/n**2*1j ]])

print(Yc)
Yt  = Ytr + Yc
print(n) 
print(Yt)

9.99493042617e-13
7.95774715459e-11
[[ 0.+0.0029531j   0.+0.000628j  ]
 [ 0.+0.000628j    0.+0.00013355j]]
4.70238390824
[[  1.00000000e-03+0.00609469j  -1.00000000e-05+0.j        ]
 [  5.00000000e-02-0.049372j     1.00000000e-04+0.00327514j]]


In [41]:
#Calculo de los Q
x = sy.Symbol('x')
ecu = sy.expand((1+x**2)*(1+(1.5*x)**2)-2)
x1 = sy.solve(ecu, x)
Q2 = x1[1]*fo/BW 
Q1 = 1.5 * Q2

print('Q1=', Q1)
print('Q2=', Q2)


#Calculo L1 en la salida suponiendo unilateral

Rext = 1/Yin_Yout(Yt)[1].real/2

print(Rext)

L1 = Rext*(1/Q2 - 1/Qo )/wo
print('L1=',L1)
print('rp=',Qo*L1*wo)

Lp = L1/n**2
print('Lp=',Lp)

Q1= 76.5817048399464
Q2= 51.0544698932976
5000.85418221
L1= 1.02833702397922e-7
rp= 9691.84811984253
Lp= 4.65049634302835e-9


In [24]:
# Calculo en la entrada usando el inductor(L1)
Rext = wo*L1/(1/Q1 - 1/Qo )
R = Rext * 2

# relacion de vueltas del generado a tope de bobina
NT1 = (R/rg)**0.5

# relacion de vueltas de la impedancia de entrada del TR a tope de bobina
rintr = 1/Yin_Yout(Yt)[0].real 
print(rintr)
NT12   = (R/rintr)**0.5

# Calculo del Capacitor de sintonia en la entrada
Cs1 = 1/(wo**2*L1)


citr = Yin_Yout(Yt)[0].imag/wo  
citrt = citr/NT12**2 
C1  = Cs1 - citrt

print('NT/N1=',NT1)
print('NT/(N1+N2)=',NT12)
print('C1=',C1)
      
# calculo de los inductores de la entrada
L1a = L1 /NT1**2      

NT2 = 1/(1/NT12 - 1/NT1)
L1b = L1 /NT2**2

NT3 = 1/(1-1/NT1-1/NT2)
L1c = L1 /NT3**2

print('L1c=',L1c)
print('L1b=',L1b)
print('L1a=',L1a)


285.687415365
NT/N1= 14.2193146661890
NT/(N1+N2)= 8.41265560115933
C1= 2.45507481376665e-11
L1c= 7.98393326072208e-8
L1b= 2.42306227992149e-10
L1a= 5.08602092041147e-10


In [27]:
# calculo del capacitor a la salida
# Calculo del Capacitor de sintonia en la entrada
Cs2 = 1/(wo**2*L1)

cotr = Yin_Yout(Yt)[1].imag/wo  
Ct  = Cs2 - cotr

# divisor capacitivo
Q  = Q2*2
rt = 1/Yin_Yout(Yt)[1].real
N  = (rt/rl)**0.5
Qm = Q/N
print('Qm=' ,Qm, '  (Si es mayor a 10 puedo aproximar)' )

# aproximando!
C3 = Ct*N/(N-1)
C2 = Ct*N

print('C2=',C2)
print('C3=',C3)

Qm= 32.2869241452235   (Si es mayor a 10 puedo aproximar)
C2= 6.14158076282448e-11
C3= 2.83997461831485e-11


In [37]:
print(1/Yin_Yout(Yt)[0].real,Yin_Yout(Yt)[0].imag/wo )
print(1/Yin_Yout(Yt)[1].real,Yin_Yout(Yt)[1].imag/wo )

285.687415365 5.7709519502e-12
10001.7083644 5.21256519805e-12
